In [1]:
#pip install numpy
#pip install pandas
#pip install ephem
#pip install plotly
#pip install PyQt5
#pip install pyqtwebengine

In [2]:
# Libraries

import ephem
import numpy as np
import pandas as pd
import plotly
import plotly.express as px
import plotly.graph_objects as go
from PyQt5 import QtCore, QtGui, QtWidgets, QtWebEngineWidgets
import os
from datetime import datetime

# Declare variables

Date = datetime.today().strftime('%Y/%m/%d')
Time = datetime.now().strftime("%H:%M:%S")
Lat = '0'
Long = '0'


# Function to calculate Azimuth, Altitude & Stellar Magnitude of every celestial body and append to table.

def calculate(): 

    # Observer

    o = ephem.Observer()
    o.lat, o.long = Lat, Long
    o.date = Date + ' ' + Time

    # List of Navigation Stars

    Stars = ['Alpheratz', 'Ankaa', 'Schedar', 'Diphda', 'Achernar', 'Hamal', 'Acamar', 'Menkar', 'Mirfak', 
             'Suhail', 'Aldebaran', 'Rigel', 'Capella', 'Bellatrix', 'Elnath', 'Alnilam', 'Betelgeuse', 
             'Canopus', 'Sirius', 'Adhara', 'Procyon', 'Pollux', 'Avior', 'Miaplacidus', 'Alphard', 'Regulus', 
             'Dubhe', 'Denebola','Gienah', 'Acrux', 'Gacrux', 'Alioth', 'Spica', 'Alkaid', 'Hadar', 'Menkent', 
             'Arcturus', 'Zubenelgenubi','Kochab', 'Alphecca', 'Antares', 'Atria', 'Sabik', 'Shaula', 
             'Rasalhague', 'Eltanin', 'Vega', 'Nunki', 'Altair', 'Peacock', 'Deneb', 'Enif', 'Fomalhaut', 
             'Markab','Polaris', 'Rigil Kentaurus','Kaus Australis', 'Alnair']
    
    # Table

    table = pd.DataFrame(columns = ['Stars', 'Azimuth', 'Altitude', 'Magnitude'])

    # Sun

    sun = ephem.Sun()
    sun.compute(o)
    alt, az, mag = sun.alt , sun.az, sun.mag
    alt, az = np.degrees(alt), np.degrees(az)
    alt, az  = np.round(alt, decimals=2), np.round(az, decimals=2)
    table.loc[len(table.index)] = ['Sun', az,alt,mag]

    # Moon

    moon = ephem.Moon()
    moon.compute(o)
    alt, az, mag = moon.alt , moon.az, moon.mag
    alt, az = np.degrees(alt), np.degrees(az)
    alt, az  = np.round(alt, decimals=2), np.round(az, decimals=2)
    table.loc[len(table.index)] = ['Moon', az,alt,mag]

    # Planets

    venus = ephem.Venus()
    venus.compute(o)
    alt, az, mag = venus.alt , venus.az, venus.mag
    alt, az = np.degrees(alt), np.degrees(az)
    alt, az  = np.round(alt, decimals=2), np.round(az, decimals=2)
    table.loc[len(table.index)] = ['Venus', az,alt,mag]

    mars = ephem.Mars()
    mars.compute(o)
    alt, az, mag = mars.alt , mars.az, mars.mag
    alt, az = np.degrees(alt), np.degrees(az)
    alt, az  = np.round(alt, decimals=2), np.round(az, decimals=2)
    table.loc[len(table.index)] = ['Mars', az,alt,mag]

    jupiter = ephem.Jupiter()
    jupiter.compute(o)
    alt, az, mag = jupiter.alt , jupiter.az, jupiter.mag
    alt, az = np.degrees(alt), np.degrees(az)
    alt, az  = np.round(alt, decimals=2), np.round(az, decimals=2)
    table.loc[len(table.index)] = ['Jupiter', az,alt,mag]

    saturn = ephem.Saturn()
    saturn.compute(o)
    alt, az, mag = saturn.alt , saturn.az, saturn.mag
    alt, az = np.degrees(alt), np.degrees(az)
    alt, az  = np.round(alt, decimals=2), np.round(az, decimals=2)
    table.loc[len(table.index)] = ['Saturn', az,alt,mag]


    # Stars

    for x in Stars:
        y = ephem.star(x)
        y.compute(o)
        alt, az, mag = y.alt , y.az, y.mag
        alt, az = np.degrees(alt), np.degrees(az)
        alt, az  = np.round(alt, decimals=2), np.round(az, decimals=2)
        table.loc[len(table.index)] = [x, az,alt,mag]

    # Size Column

    table['Size'] = table.Magnitude.map(lambda x: '40' if x < -20 else( '15' if x >= -20 and x < -5 
                                                                       else ('2' if x >= -5 and x < 0 
                                                                           else ('1.5' if x >= 0 and x < 1
                                                                                 else('1.0' if x >= 1 and x < 2 
                                                                                     else '0.5')))))
    table['Size'] = table['Size'].astype(float)

    # Symbol Column

    table['Symbol'] = 'star'
    for x in ['Sun', 'Moon', 'Venus', 'Mars', 'Jupiter', 'Saturn']:
        table.loc[(table.Stars == x), 'Symbol'] = 'circle'

    # Color Column

    table['Color'] = 'Blue'
    table.loc[(table.Stars == 'Sun'), 'Color'] = 'DarkOrange'
    table.loc[(table.Stars == 'Moon'), 'Color'] = 'Gainsboro'
    table.loc[(table.Stars == 'Mars'), 'Color'] = 'DarkRed'
    for x in ['Venus', 'Jupiter', 'Saturn']:
        table.loc[(table.Stars == x), 'Color'] = 'DarkSlateBlue'

    # Plot
    
    fig = px.scatter(table, x = 'Azimuth', y= 'Altitude', size = 'Size', size_max= 40, text = 'Stars', 
                     hover_name = 'Stars', hover_data = {'Size': False, 'Stars':False}, symbol = 'Symbol', 
                     symbol_map= 'identity', color = 'Color', color_discrete_map= 'identity', 
                     )

    fig.update_yaxes(range=[0, 90], fixedrange =True, showgrid = False)
    fig.update_xaxes(range=[0, 360],fixedrange=True, showgrid = False)
    fig.update_traces(textposition='top center', marker=dict(line=dict(width=0)))

    if table.loc[0,'Altitude'] > 10:
        fig.update_layout(plot_bgcolor= 'lemonchiffon')
        fig.update_traces(textfont=dict(family="sans serif",color='black'))
    elif table.loc[0,'Altitude'] < -5:
        fig.update_layout(plot_bgcolor= 'black')
        fig.update_traces(textfont=dict(family="sans serif",color="teal"))
    else:
        fig.update_layout(plot_bgcolor= 'lightsteelblue')
        fig.update_traces(textfont=dict(family="sans serif",color='snow'))

    #fig.show()
    fig.write_html('plot.html')
    table.head()
    
calculate()    

In [3]:
# GUI

class Ui_MainWindow(object):
    def setupUi(self, MainWindow):
        MainWindow.setObjectName("MainWindow")
        MainWindow.resize(1280, 772)
        sizePolicy = QtWidgets.QSizePolicy(QtWidgets.QSizePolicy.Fixed, QtWidgets.QSizePolicy.Fixed)
        sizePolicy.setHorizontalStretch(0)
        sizePolicy.setVerticalStretch(0)
        sizePolicy.setHeightForWidth(MainWindow.sizePolicy().hasHeightForWidth())
        MainWindow.setSizePolicy(sizePolicy)
        self.centralwidget = QtWidgets.QWidget(MainWindow)
        self.centralwidget.setObjectName("centralwidget")
        self.splitter = QtWidgets.QSplitter(self.centralwidget)
        self.splitter.setGeometry(QtCore.QRect(40, 10, 1200, 134))
        self.splitter.setOrientation(QtCore.Qt.Vertical)
        self.splitter.setObjectName("splitter")
        self.label_9 = QtWidgets.QLabel(self.splitter)
        font = QtGui.QFont()
        font.setFamily("Arial Black")
        font.setPointSize(40)
        font.setBold(True)
        font.setUnderline(True)
        font.setWeight(75)
        self.label_9.setFont(font)
        self.label_9.setAlignment(QtCore.Qt.AlignCenter)
        self.groupBox = QtWidgets.QGroupBox(self.splitter)
        sizePolicy = QtWidgets.QSizePolicy(QtWidgets.QSizePolicy.Preferred, QtWidgets.QSizePolicy.Preferred)
        sizePolicy.setHorizontalStretch(0)
        sizePolicy.setVerticalStretch(0)
        sizePolicy.setHeightForWidth(self.groupBox.sizePolicy().hasHeightForWidth())
        self.groupBox.setSizePolicy(sizePolicy)
        self.groupBox.setMaximumSize(QtCore.QSize(1200, 80))
        font = QtGui.QFont()
        font.setFamily("Arial")
        font.setPointSize(15)
        font.setWeight(50)
        self.gridLayout = QtWidgets.QGridLayout(self.groupBox)
        
        # Date Lable
        self.label = QtWidgets.QLabel(self.groupBox)
        self.gridLayout.addWidget(self.label, 0, 0, 1, 1)
        
        # Date
        self.dateEdit = QtWidgets.QDateEdit(self.groupBox)
        self.dateEdit.setDateTime(QtCore.QDateTime.currentDateTime())
        self.dateEdit.setCalendarPopup(True)
        self.gridLayout.addWidget(self.dateEdit, 0, 1, 1, 1)
        
        # Time Lable
        self.label_2 = QtWidgets.QLabel(self.groupBox)
        self.gridLayout.addWidget(self.label_2, 0, 2, 1, 1)
        
        # Time
        self.timeEdit = QtWidgets.QTimeEdit(self.groupBox)
        self.timeEdit.setDateTime(QtCore.QDateTime.currentDateTimeUtc())
        self.gridLayout.addWidget(self.timeEdit, 0, 3, 1, 1)
        
        #Lat Lable
        self.label_3 = QtWidgets.QLabel(self.groupBox)
        self.gridLayout.addWidget(self.label_3, 0, 4, 1, 1)
        
        # Lat 1
        self.lineEdit = QtWidgets.QLineEdit(self.groupBox)
        self.lineEdit.setLayoutDirection(QtCore.Qt.LeftToRight)
        self.lineEdit.setMaxLength(2)
        self.lineEdit.setAlignment(QtCore.Qt.AlignRight|QtCore.Qt.AlignTrailing|QtCore.Qt.AlignVCenter)
        self.gridLayout.addWidget(self.lineEdit, 0, 5, 1, 1)
        
        # deg lable
        self.label_5 = QtWidgets.QLabel(self.groupBox)
        self.gridLayout.addWidget(self.label_5, 0, 6, 1, 1)
        
        # Lat 2
        self.lineEdit_2 = QtWidgets.QLineEdit(self.groupBox)
        self.lineEdit_2.setMaxLength(5)
        self.lineEdit_2.setAlignment(QtCore.Qt.AlignRight|QtCore.Qt.AlignTrailing|QtCore.Qt.AlignVCenter)
        self.gridLayout.addWidget(self.lineEdit_2, 0, 7, 1, 1)
        
        # min lable
        self.label_6 = QtWidgets.QLabel(self.groupBox)
        self.gridLayout.addWidget(self.label_6, 0, 8, 1, 1)
        
        # N/S
        self.comboBox = QtWidgets.QComboBox(self.groupBox)
        self.comboBox.setMinimumSize(QtCore.QSize(55, 0))
        self.comboBox.addItem("")
        self.comboBox.addItem("")
        self.gridLayout.addWidget(self.comboBox, 0, 9, 1, 1)
        
        # Longitude lable
        self.label_4 = QtWidgets.QLabel(self.groupBox)
        self.gridLayout.addWidget(self.label_4, 0, 10, 1, 1)
        
        # Long 1
        self.lineEdit_3 = QtWidgets.QLineEdit(self.groupBox)
        self.lineEdit_3.setMaxLength(3)
        self.lineEdit_3.setAlignment(QtCore.Qt.AlignRight|QtCore.Qt.AlignTrailing|QtCore.Qt.AlignVCenter)
        self.gridLayout.addWidget(self.lineEdit_3, 0, 11, 1, 1)
        
        # deg
        self.label_8 = QtWidgets.QLabel(self.groupBox)
        self.gridLayout.addWidget(self.label_8, 0, 12, 1, 1)
        
        # Long 2
        self.lineEdit_4 = QtWidgets.QLineEdit(self.groupBox)
        self.lineEdit_4.setMaxLength(5)
        self.lineEdit_4.setAlignment(QtCore.Qt.AlignRight|QtCore.Qt.AlignTrailing|QtCore.Qt.AlignVCenter)
        self.gridLayout.addWidget(self.lineEdit_4, 0, 13, 1, 1)
        
        # min
        self.label_7 = QtWidgets.QLabel(self.groupBox)
        self.gridLayout.addWidget(self.label_7, 0, 14, 1, 1)
        
        # E/W
        self.comboBox_2 = QtWidgets.QComboBox(self.groupBox)
        self.comboBox_2.setMinimumSize(QtCore.QSize(55, 0))
        self.comboBox_2.addItem("")
        self.comboBox_2.addItem("")
        self.gridLayout.addWidget(self.comboBox_2, 0, 15, 1, 1)
        
        #Push Button
        self.pushButton = QtWidgets.QPushButton(self.groupBox)
        self.gridLayout.addWidget(self.pushButton, 0, 16, 1, 1)
        self.pushButton.clicked.connect(self.click)
        
        # Signature
        self.label_10 = QtWidgets.QLabel(self.centralwidget)
        self.label_10.setGeometry(540,700,500,100)
        self.label_10.setText('Created by - Nishchay Joshi')
        font.setBold(True)
        self.label_10.setFont(font)

        MainWindow.setCentralWidget(self.centralwidget)
        self.menubar = QtWidgets.QMenuBar(MainWindow)
        self.menubar.setGeometry(QtCore.QRect(0, 0, 1280, 22))
        MainWindow.setMenuBar(self.menubar)
        self.statusbar = QtWidgets.QStatusBar(MainWindow)
        MainWindow.setStatusBar(self.statusbar)

        self.retranslateUi(MainWindow)
        QtCore.QMetaObject.connectSlotsByName(MainWindow)
        
    def plot(self, MainWindow):
        self.webEngineView = QtWebEngineWidgets.QWebEngineView(self.centralwidget)
        self.webEngineView.setGeometry(QtCore.QRect(40, 160, 1201, 541))

    def retranslateUi(self, MainWindow):
        _translate = QtCore.QCoreApplication.translate
        MainWindow.setWindowTitle(_translate("MainWindow", "Star Finder"))
        self.label_9.setText(_translate("MainWindow", "STAR FINDER"))
        self.label.setText(_translate("MainWindow", "Date (dd/mm/yy):"))
        self.label_2.setText(_translate("MainWindow", "Time (UTC):"))
        self.label_3.setText(_translate("MainWindow", "Latitude:"))
        self.lineEdit.setText(_translate("MainWindow", "29"))
        self.label_5.setText(_translate("MainWindow", "deg"))
        self.lineEdit_2.setText(_translate("MainWindow", "13.10"))
        self.label_6.setText(_translate("MainWindow", "min"))
        self.comboBox.setItemText(0, _translate("MainWindow", "N"))
        self.comboBox.setItemText(1, _translate("MainWindow", "S"))
        self.label_4.setText(_translate("MainWindow", "Longitude:"))
        self.lineEdit_3.setText(_translate("MainWindow", "079"))
        self.label_8.setText(_translate("MainWindow", "deg"))
        self.lineEdit_4.setText(_translate("MainWindow", "30.78"))
        self.label_7.setText(_translate("MainWindow", "min"))
        self.comboBox_2.setItemText(0, _translate("MainWindow", "E"))
        self.comboBox_2.setItemText(1, _translate("MainWindow", "W"))
        self.pushButton.setText(_translate("MainWindow", "Calculate"))
    
    def click(self):
        global Lat, Long, Date, Time
        
        # Latitude input
        deg = float(self.lineEdit.text())
        mint = float(self.lineEdit_2.text())
        lat = deg + mint/60
        if self.comboBox.currentText() == 'N':
            Lat = lat
        else:
            Lat = lat * -1
        Lat = str(round(Lat, 3))
        
        # Longitude input
        deg = float(self.lineEdit_3.text())
        mint = float(self.lineEdit_4.text())
        long = deg + mint/60
        if self.comboBox_2.currentText() == 'E':
            Long = long
        else:
            Long = long * -1
        Long = str(round(Long, 3))
        
        # Date input
        Date = self.dateEdit.date().toString('yyyy/MM/dd')
        
        # Time input
        Time = self.timeEdit.time().toString()
        
        calculate()
        self.webEngineView.load(QtCore.QUrl.fromLocalFile(os.path.abspath('plot.html')))
 
            
if __name__ == "__main__":
    import sys
    app = QtWidgets.QApplication(sys.argv)
    MainWindow = QtWidgets.QMainWindow()
    ui = Ui_MainWindow()
    ui.setupUi(MainWindow)
    ui.plot(MainWindow)
    MainWindow.show()
    sys.exit(app.exec_())

SystemExit: 0

/opt/anaconda3/envs/NaAl/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3452: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.

